In [ ]:
"""
    Import all the required libraries to be able to classify humans 
    spotted on images into the right class.
    
"""

from astral import LocationInfo
from astral.sun import sun
from datetime import datetime
import pytz
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
from PIL import Image, ExifTags

In [4]:
"""
    Load the most recent version of the YOLOv5 model. This model gets used to predict
    humans who are visible within the photo: img.
    
"""

model = torch.hub.load('ultralytics/yolov5', 'custom', 
                            path=os.path.join("yolov5","runs","train","exp30","weights","best.pt"), force_reload=True)
img = os.path.join("C","Users","rdede", "Downloads", "0000103_02544_d_0000029.jpg")
results=model(img)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\rdede/.cache\torch\hub\master.zip
YOLOv5  2022-5-12 torch 1.11.0+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7015519 parameters, 0 gradients
Adding AutoShape... 


In [2]:
"""
    Making variables which are necessary to classify humans correctly based on time and location.
    
    --------
    
    city - Location information, which gives back the timezone and coordinates.
    s - Sun information, gives back the position of the sun and what times rise and dawn are.
    created_datetime - Picture information, gives back the time the photo has been created.
                        *if pictures don't have relevant metadata, current time is used*
    image - Read and converted image into Numpy Array.
    exif_data - Image metadata, gives back all the metadata included with the image.
    
"""

city = LocationInfo("Amsterdam", "Netherlands", "Europe/Amsterdam", 52.378, 4.897)
s = sun(city.observer, date=datetime.date.today(), tzinfo=pytz.timezone(city.timezone))
created_datetime = datetime.now(pytz.timezone(city.timezone))
image = Image.open(img)
exif_data = image._getexif()

for key, val in exif_data.items():
    if key == 36868:
        created_datetime = val
        created_datetime = datetime.strptime(created_datetime, '%Y:%m:%d %H:%M:%S')
        tz = pytz.timezone(city.timezone)
        created_datetime = created_datetime.replace(tzinfo=tz)

Dawn:    2022-05-12 05:07:32.934710+02:00
Sunrise: 2022-05-12 05:50:15.171836+02:00
Noon:    2022-05-12 13:36:47+02:00
Sunset:  2022-05-12 21:24:20.148728+02:00
Dusk:    2022-05-12 22:07:20.489627+02:00



In [10]:
"""
    Showing the image with it's bounding boxes. Around the bounding box, the class 
    of the person is given.
    
    --------
    
    During daytime:
      - People who are detected and classified as authorised are shown as authorised.
      - People who are detected and classified as non-authorised are shown as authorised.
     
     During nighttime:
      - People who are detected and classified as authorised are shown as authorised.
      - People who are detected and classified as non-authorised as shown as non-authorised.
    
"""

for box in results.xyxy[0]: 

    xB = int(box[2])
    xA = int(box[0])
    yB = int(box[3])
    yA = int(box[1])
    
    image = cv2.rectangle(image, (xA, yA), (xB, yB), (255, 0, 0), 2)

    if (created_datetime > s['sunset'] or created_datetime < s['sunrise']):
        print("It's nighttime")
        if int(box[5]) == 1:
            org = (xA,yA-10)
            image = cv2.putText(image, 'Not Authorised', org, cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        if int(box[5]) == 0:
            org = (xA,yA-10)
            image = cv2.putText(image, 'Authorised', org, cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    else:
        print("It's daytime")
        org = (xA,yA-10)
        image = cv2.putText(image, 'Authorised', org, cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
image = cv2.resize(image, (960, 540))
cv2.imshow("YOLO", image)
cv2.waitKey(0)

It's nighttime
It's nighttime
It's nighttime
It's nighttime
It's nighttime


-1